<a href="https://colab.research.google.com/github/EmmaLee01/SASL-E-Sports/blob/branch_Elias/SASL_Trial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests as rq
import json

myAPI = "foE5KBNApBF2PKMt19jmSjKsHsnytXINHXG79Aoa"  # Replace with your actual API key

headers = {
    "x-api-key": myAPI,
    "Content-Type": "application/json"
}

SERIES_STATE_ENDPOINT = "https://api-op.grid.gg/live-data-feed/series-state/graphql"

# ============================================================
# Introspection for PlayerInventory -> ItemStack
# ============================================================
type_name = "ItemStack"
introspection_query = f"""
{{
  __type(name: "{type_name}") {{
    name
    fields {{
      name
      type {{
        name
        kind
        ofType {{
          name
          kind
        }}
      }}
    }}
  }}
}}
"""

payload = {"query": introspection_query}
response = rq.post(SERIES_STATE_ENDPOINT, headers=headers, json=payload, timeout=10)

print("\n" + "="*60)
print(f"FIELDS IN {type_name}")
print("="*60)

if response.status_code == 200:
    data = response.json()
    if 'data' in data and '__type' in data['data'] and data['data']['__type']:
        fields = data['data']['__type']['fields']
        if fields:
            for field in fields:
                field_type = field['type']
                if field_type.get('name'):
                    type_info = field_type['name']
                elif field_type.get('ofType'):
                    type_info = field_type['ofType'].get('name', 'Unknown')
                else:
                    type_info = field_type.get('kind', 'Unknown')
                print(f"  • {field['name']}: {type_info}")
        else:
            print("  No fields (might be a scalar)")
    else:
        print("  Type not found or is a scalar")
else:
    print(f"❌ Request failed for {type_name}: {response.status_code} – {response.text}")

# ============================================================
# Main Query: series state with minimal inventory items
# ============================================================
print("\n" + "="*60)
print("Running series state query with minimal inventory items")
print("="*60)

graphql_query = """
query GetSeriesStateWithInventory($seriesId: ID!) {
  seriesState(id: $seriesId) {
    id
    version
    format
    started
    finished
    valid
    updatedAt
    startedAt
    duration
    title {
      nameShortened
    }
    teams {
      id
      name
    }
    games {
      id
      sequenceNumber
      type
      started
      finished
      paused
      startedAt
      duration
      clock {
        id
        type
        ticking
        ticksBackwards
        currentSeconds
      }
      map {
        name
      }
      teams {
        id
        name
        side
        won
        score
        money
        loadoutValue
        kills
        deaths
        players {
          id
          name
          money
          loadoutValue
          kills
          deaths
          killAssistsReceived
          killAssistsGiven
          position {
            x
            y
          }
          inventory {
            items {
              id
              name
            }
          }
        }
      }
      segments {
        id
        type
        sequenceNumber
        started
        finished
        startedAt
        duration
      }
    }
  }
}
"""

variables = {"seriesId": "28"}  # Change the series ID as needed

payload = {
    "query": graphql_query,
    "variables": variables
}

response = rq.post(SERIES_STATE_ENDPOINT, headers=headers, json=payload, timeout=10)

if response.status_code == 200:
    data = response.json()

    if 'errors' in data:
        print("⚠️  Errors occurred:")
        print(json.dumps(data['errors'], indent=2))

    if 'data' in data:
        print("\n✅ SUCCESS! Series State Data with Inventory (minimal):")
        print(json.dumps(data, indent=2))

        series = data['data']['seriesState']
        print("\n" + "="*60)
        print(f"Series {series['id']} - {series['format']}")
        print(f"Status: {'Finished' if series['finished'] else 'Ongoing'}")
        print(f"Duration: {series['duration']}")
        print("="*60)

        for game in series.get('games', []):
            print(f"\n🎮 Game {game.get('sequenceNumber', '?')}:")
            print(f"   Status: {'Finished' if game.get('finished') else 'Ongoing'}")

            if game.get('clock'):
                clock = game['clock']
                direction = "⬇️ Counting down" if clock['ticksBackwards'] else "⬆️ Counting up"
                status = "🟢 Ticking" if clock['ticking'] else "⏸️ Paused"
                print(f"   ⏰ Clock: {clock['currentSeconds']}s ({direction}, {status})")
                print(f"   Clock Type: {clock['type']}")

            for team in game.get('teams', []):
                print(f"\n   Team: {team['name']} ({team.get('side', 'N/A')})")
                print(f"   Score: {team.get('score', 0)} | K/D: {team.get('kills', 0)}/{team.get('deaths', 0)}")
                print(f"   Money: ${team.get('money', 0):,}")

                for player in team.get('players', []):
                    print(f"      Player: {player['name']} | Money: ${player['money']} | Loadout: ${player['loadoutValue']}")
                    inv_items = player.get("inventory", {}).get("items", [])
                    if inv_items:
                        for item in inv_items:
                            print(f"         • Item ID: {item.get('id')} | Name: {item.get('name')}")
else:
    print(f"❌ Request failed: {response.status_code}")
    print(response.text)



FIELDS IN ItemStack
  • id: ID
  • name: String
  • quantity: Int
  • equipped: Int
  • stashed: Int

Running series state query with minimal inventory items

✅ SUCCESS! Series State Data with Inventory (minimal):
{
  "data": {
    "seriesState": {
      "id": "28",
      "version": "3.47",
      "format": "best-of-3",
      "started": true,
      "finished": false,
      "valid": true,
      "updatedAt": "2025-10-28T13:30:46.543Z",
      "startedAt": "2025-10-28T13:02:28.971Z",
      "duration": "PT28M17.572S",
      "title": {
        "nameShortened": "cs2"
      },
      "teams": [
        {
          "id": "52314",
          "name": "CS2-1"
        },
        {
          "id": "52315",
          "name": "CS2-2"
        }
      ],
      "games": [
        {
          "id": "6d183957-c4a6-42cf-ab04-dd72e42e108a",
          "sequenceNumber": 1,
          "type": "ESPORTS",
          "started": true,
          "finished": false,
          "paused": false,
          "startedAt": "2025-